In [ ]:
import pandas as pd
import numpy as np
import random

import openai
import os
from tqdm import tqdm
import time

import sys

sys.path.insert(0, '..')

In [ ]:
# TURBO COST
TURBO_COST = 0.00200 / 1000

In [ ]:
# Initialize the API client
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
df = pd.read_csv('../data/seed/youtube/seed_videos.csv')

In [ ]:
### BASE PROMPT
# Labels:

# - Blockchain: Cryptocurrency, NFTs, or other Web3 investments
# - Traditional: Stocks, Bonds, Real Estate, Commodities, or other traditional investments
# - Mixed: Videos that recommend or educate about both blockchain and traditional investments
# - None: Videos that do not recommend or educate about either blockchain or traditional investments

# Instructions:

# - If a video snippet mentions keywords related to blockchain or traditional investments but does not recommend or educate about the label, it will be classified as None.
# - Please return predictions in the format" {Label} : {50 word or shorter rationale}"""},
#         {"role": "assistant", "content": """Understood. I will classify YouTube video snippets based on the provided labels and instructions. Here's how I will format the predictions:

# {Label} : {50-word or shorter rationale}

# Please provide me with the YouTube video snippet you would like me to classify.

In [ ]:
### Deal with failure to classify retirement accounts
# Labels:

# - Blockchain: Cryptocurrency, NFTs, or other Web3 investments
# - Traditional: Stocks, Bonds, Real Estate, Commodities, Retirement Accounts
# - Mixed: Videos that recommend or educate about both blockchain and traditional investments
# - None: Videos that do not recommend or educate about either blockchain or traditional investments

# Instructions:

# - If a video snippet mentions keywords related to blockchain or traditional investments but does not recommend or educate about the label, it will be classified as None.
# - Please return predictions in the format" {Label} : {50 word or shorter rationale}"""},
#         {"role": "assistant", "content": """Understood. I will classify YouTube video snippets based on the provided labels and instructions. Here's how I will format the predictions:

# {Label} : {50-word or shorter rationale}

# Please provide me with the YouTube video snippet you would like me to classify.

In [ ]:
### Better accuracy failed other things
# Labels:

# Traditional: Videos that recommend or educate about stocks, bonds, real estate, commodities, retirement accounts, or other traditional investments or keywords related to them.
# Blockchain: Videos that recommend or educate about cryptocurrency, NFTs, or other Web3 investments or keywords related to them.
# Mixed: Videos that recommend or educate about both blockchain and traditional investments or keywords related to both.
# Unrelated: Videos that do not recommend or educate about either blockchain or traditional investments or keywords related to them.

# Instructions:
# If a video snippet mentions any of the keywords related to traditional or blockchain investments, it will be classified accordingly. The classifier should consider the context and meaning of the keywords used to determine whether the snippet is related to traditional or blockchain investments.

In [ ]:
### High accuracy, great precision recall on traditional and blockchain, really bad at None/Unrelated
# Labels:

# Traditional: Videos that recommend or educate about stocks, bonds, real estate, commodities, retirement accounts, or other traditional investments or keywords related to them.
# Blockchain: Videos that recommend or educate about cryptocurrency, NFTs, or other Web3 investments or keywords related to them.
# Mixed: Videos that recommend or educate about both blockchain and traditional investments or keywords related to both.
# Unrelated: Videos that do not recommend or educate about either blockchain or traditional investments or keywords related to them.

# Instructions:
#  - If a video snippet mentions any of the keywords related to traditional or blockchain investments, it will be classified accordingly. The classifier should consider the context and meaning of the keywords used to determine whether the snippet is related to traditional or blockchain investments.
#  - Please return predictions in the format" {Label} : {20 word or shorter rationale}

In [ ]:
def create_messages(snippet):
    messages = [
        {"role": "system", "content" : "You are a classifier that determines if a YouTube video snippet falls under a label. A snippet is a concatenation of the video title, summarized transcript, and video tags. The labels and additional instructions will be included in the first user message."},
        {"role": "user", "content" : """Labels:

Traditional: Videos that recommend or educate about stocks, bonds, real estate, commodities, retirement accounts, or other traditional investments or keywords related to them.
Blockchain: Videos that recommend or educate about cryptocurrency (BTC, ETH, etc.), NFTs, or other Web3 investments or keywords related to them.
Mixed: Videos that recommend or educate about both blockchain and traditional investments or keywords related to both.
Unrelated: Videos that do not recommend or educate about either blockchain or traditional investments or keywords related to them.

Instructions:
- The classifier should consider the context and meaning of the keywords used to determine whether the snippet is related to traditional or blockchain investments.
- If talks about making money from jobs, side hustles, or other alternative assets (cars, watches, artificial intelligence, trading cards, art, etc), they are Unrelated.
- If a video is only discussing the asset(s) negatively, it should be classified as Unrelated.
- Please return predictions in the format" {Label} : {20 word or shorter rationale}"""},
        {"role": "assistant", "content": """Understood. I will classify YouTube video snippets based on the provided labels and instructions. Here's how I will format the predictions:

{Label} : {20-word or shorter rationale}

Please provide me with the YouTube video snippet you would like me to classify."""},
    ]
    
    snippet_message = {"role" : "user", "content" : snippet.replace("\n", " ").replace("  ", " ")}
    
    messages.append(snippet_message)
    
    return messages

In [ ]:
def classify(messages):
    chatCompletion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=0.25
    )
    return chatCompletion

In [ ]:
df['messages'] = df['snippet'].apply(create_messages)

In [ ]:
completions = []
completion_df = []


pbar = tqdm(tough_traditional.iterrows())  

for idx, entry in pbar:
    pbar.set_description("Processing %s" % entry['title'])
    
    body = classify(entry['messages'])
    
    ## Add error catching
    completion = dict(body['choices'][0]['message'])
    completion['prediction']  = completion['content'].split(':')[0].strip()
    completion['reason'] = completion['content'].split(':')[1].strip()
    
    ## Grab meta data
    completion.update({key: body[key] for key in ['created', 'id', 'model', 'object']})
    
    ## Grab token usage
    completion.update(dict(body['usage']))

    completions.append(completion)
    
    pd.DataFrame(completions).to_csv('seed_predictions.csv', index_label = False)
    
    
    time.sleep(10)

In [ ]:
predictions = pd.read_csv('seed_predictions.csv')

In [ ]:
for reason in predictions['reason']:
    print(reason + '\n')

In [ ]:
sum(predictions['total_tokens']) * TURBO_COST

In [ ]:
predictions['prediction']

In [ ]:
print(tough_traditional['messages'][34][3]['content'])

In [ ]:
print(df['messages'][0][1]['content'])

In [ ]:
completions = []
completion_df = []

pbar = tqdm(df.iterrows())  

for idx, entry in pbar:
    pbar.set_description("Processing %s" % entry['title'])
    
    body = classify(entry['messages'])
    
    ## Add error catching
    completion = dict(body['choices'][0]['message'])
    completion['prediction']  = completion['content'].split(':')[0].strip()
    completion['reason'] = completion['content'].split(':')[1].strip()
    completion['message'] = entry['messages']
    completion['title'] = entry['title']
    completion['transcript'] = entry['cleaned_transcript']
    completion['snippet'] = entry['snippet']
    completion['link'] = entry['link']

    ## Grab meta data
    completion.update({key: body[key] for key in ['created', 'id', 'model', 'object']})
    
    ## Grab token usage
    completion.update(dict(body['usage']))

    completions.append(completion)
    
    pd.DataFrame(completions).to_csv('seed_predictions.csv', index_label = False)
    
    
    time.sleep(5)

In [ ]:
predictions = pd.read_csv('seed_predictions.csv')

In [ ]:
preds = list(predictions['prediction'].apply(lambda x: 'None' if x == 'Unrelated' else x))
actual = list(df['label'])
preds = ['Blockchain' if pred == 'Label' else pred for pred in preds]
sum(predictions['total_tokens']) * TURBO_COST

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy_score(actual, preds)

In [ ]:
print(classification_report(actual, preds))

In [ ]:
predictions

In [ ]:
df['label'][70:81]

In [ ]:
print(df.iloc[23]['snippet'])

In [ ]:
predictions.iloc[23]['reason']

In [ ]:
### This should not be Mixed, it should be traditional. It just has video tags that mentions crypto

In [ ]:
print(df.iloc[54]['messages'][3]['content'])

In [ ]:
predictions.iloc[54]['reason']

In [ ]:
### This should be blockchain, not mixed. Shift focus to when it's mainly talking about bitcoin

In [ ]:
print(df.iloc[65]['messages'][3]['content'])

In [ ]:
predictions.iloc[65]['reason']

In [ ]:
### BTC is not an investment, maybe provide examples of bitcoin ticker symbols

In [ ]:
print(df.iloc[81]['messages'][3]['content'])

In [ ]:
print(predictions.iloc[81]['reason'])

In [ ]:
### Speculating on bitcoin.

In [ ]:
print(df.iloc[82]['snippet'])

In [ ]:
print(predictions.iloc[82]['reason'])

In [ ]:
print(df.iloc[83]['messages'][3]['content'])

In [ ]:
print(predictions.iloc[83]['reason'])

In [ ]:
print(df.iloc[84]['messages'][3]['content'])

In [ ]:
print(predictions.iloc[84]['reason'])

In [ ]:
# Arguably, 85 is mislabelled by me hahaha...

In [ ]:
print(df.iloc[85]['messages'][3]['content'])

In [ ]:
print(predictions.iloc[85]['reason'])

In [ ]:
print(df.iloc[86]['messages'][3]['content'])

In [ ]:
print(predictions.iloc[86]['reason'])

In [ ]:
print(df.iloc[87]['messages'][3]['content'])

In [ ]:
print(predictions.iloc[87]['reason'])

In [ ]:
print(df.iloc[88]['messages'][3]['content'])

In [ ]:
print(predictions.iloc[88]['reason'])

In [ ]:
print(df.iloc[89]['messages'][3]['content'])

In [ ]:
print(predictions.iloc[89]['reason'])

In [ ]:
print(df.iloc[90]['snippet'])

In [ ]:
print(predictions.iloc[90]['reason'])

In [ ]:
print(df.iloc[91]['messages'][3]['content'])

In [ ]:
print(predictions.iloc[91]['reason'])

In [ ]:
print(df.iloc[92]['snippet'])

In [ ]:
print(predictions.iloc[92]['reason'])

In [ ]:
## I can argue this as a traditional video

In [ ]:
print(df.iloc[93]['messages'][3]['content'])

In [ ]:
print(predictions.iloc[93]['reason'])

In [ ]:
## I can argue this as a traditional video

In [ ]:
print(df.iloc[94]['messages'][3]['content'])

In [ ]:
print(predictions.iloc[94]['reason'])

In [ ]:
print(df.iloc[95]['messages'][3]['content'])

In [ ]:
print(predictions.iloc[95]['reason'])

In [ ]:
# Labels:

# Traditional: Videos that recommend or educate about stocks, bonds, real estate, commodities, retirement accounts, or other traditional investments or keywords related to them.
# Blockchain: Videos that recommend or educate about cryptocurrency (BTC, ETH, etc.), NFTs, or other Web3 investments or keywords related to them.
# Mixed: Videos that recommend or educate about both blockchain and traditional investments or keywords related to both.
# Unrelated: Videos that do not recommend or educate about either blockchain or traditional investments or keywords related to them.

# Instructions:
# - The classifier should consider the context and meaning of the keywords used to determine whether the snippet is related to traditional or blockchain investments.
# - If talks about making money from jobs, side hustles, or other alternative assets (cars, watches, artificial intelligence, trading cards, art, etc), they are Unrelated.
# - A video that is downplaying an investment or discussing it negatively should be classified as Unrelated.
# - Please return predictions in the format" {Label} : {20-word or shorter rationale

In [ ]:
completions = []
completion_df = []

pbar = tqdm(df.iterrows())  

for idx, entry in pbar:
    pbar.set_description("Processing %s" % entry['title'])
    
    body = classify(entry['messages'])
    
    ## Add error catching
    completion = dict(body['choices'][0]['message'])
    completion['prediction']  = completion['content'].split(':')[0].strip()
    completion['reason'] = completion['content'].split(':')[1].strip()
    completion['message'] = entry['messages']
    completion['title'] = entry['title']
    completion['transcript'] = entry['cleaned_transcript']
    completion['snippet'] = entry['snippet']
    completion['link'] = entry['link']

    ## Grab meta data
    completion.update({key: body[key] for key in ['created', 'id', 'model', 'object']})
    
    ## Grab token usage
    completion.update(dict(body['usage']))

    completions.append(completion)
    
    pd.DataFrame(completions).to_csv('seed_predictions.csv', index_label = False)
    
    
    time.sleep(5)

In [ ]:
predictions = pd.read_csv('seed_predictions.csv')

In [ ]:
preds = list(predictions['prediction'].apply(lambda x: 'None' if x == 'Unrelated' else x))
actual = list(df['label'])
preds = ['Blockchain' if pred == 'Label' else pred for pred in preds]
sum(predictions['total_tokens']) * TURBO_COST

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy_score(actual, preds)

In [ ]:
print(classification_report(actual, preds))

In [ ]:
predictions

In [ ]:
predictions.iloc[45]['reason']
# Predicted mixed because the recommender was skeptical, thoiugh the title says coin will skyrocket, adjust prompt accordingly

In [ ]:
predictions.iloc[45]['snippet']

In [ ]:
predictions.iloc[64]['reason']
# impact of social media led to this being predicted as mixed, also its sus about unrelated terminology like (like and subscribe)

In [ ]:
print(predictions.iloc[64]['snippet'])

In [ ]:
predictions.iloc[65]['reason']
# classifying BTC as traditional, talks about side hustles and other information as well

In [ ]:
print(predictions.iloc[65]['snippet'])

In [ ]:
# Labels:

# Traditional: Videos that recommend or educate about stocks, bonds, real estate, commodities, retirement accounts, or other traditional investments or keywords related to them.
# Blockchain: Videos that recommend or educate about cryptocurrency (BTC, ETH, etc.), NFTs, or other Web3 investments or keywords related to them.
# Mixed: Videos that recommend or educate about both blockchain and traditional investments or keywords related to both.
# Unrelated: Videos that do not recommend or educate about either blockchain or traditional investments or keywords related to them.

# Instructions:
# - The classifier should consider the context and meaning of the keywords used to determine whether the snippet is related to traditional or blockchain investments.
# - If talks about making money from jobs, side hustles, or other alternative assets (cars, watches, artificial intelligence, trading cards, art, etc), they are Unrelated.
# - If a video is only discussing the asset negatively, it should be classified as Unrelated.
# - Please return predictions in the format" {Label} : {20 word or shorter rationale}

In [ ]:
completions = []
completion_df = []

pbar = tqdm(df.iterrows())  

for idx, entry in pbar:
    pbar.set_description("Processing %s" % entry['title'])
    
    body = classify(entry['messages'])
    
    ## Add error catching
    completion = dict(body['choices'][0]['message'])
    completion['prediction']  = completion['content'].split(':')[0].strip()
    completion['reason'] = completion['content'].split(':')[1].strip()
    completion['message'] = entry['messages']
    completion['title'] = entry['title']
    completion['transcript'] = entry['cleaned_transcript']
    completion['snippet'] = entry['snippet']
    completion['link'] = entry['link']

    ## Grab meta data
    completion.update({key: body[key] for key in ['created', 'id', 'model', 'object']})
    
    ## Grab token usage
    completion.update(dict(body['usage']))

    completions.append(completion)
    
    pd.DataFrame(completions).to_csv('seed_predictions.csv', index_label = False)
    
    
    time.sleep(5)

In [ ]:
predictions = pd.read_csv('seed_predictions.csv')

In [ ]:
preds = list(predictions['prediction'].apply(lambda x: 'None' if x == 'Unrelated' else x))
actual = list(df['label'])
preds = ['Blockchain' if pred == 'Label' else pred for pred in preds]
sum(predictions['total_tokens']) * TURBO_COST

In [ ]:
accuracy_score(actual, preds)

In [ ]:
print(classification_report(actual, preds))

In [ ]:
predictions

In [ ]:
print(predictions.iloc[0]['snippet'])